In [2]:
# 설치가 필요한 패키지
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests

# 파이썬에 내장되어 있는 패키지
import datetime as dt
import os
import zipfile   
import shutil
import time

In [3]:
options = webdriver.ChromeOptions()
mobile_emulation = {"deviceName": "Nexus 5"}
options.add_experimental_option("mobileEmulation", mobile_emulation)

# 크롬 브라우저를 백그라운드 프로세스 형태로 실행시키고자 하는 경우 아래의 옵션도 추가한다.
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")

## 크롬 브라우저 띄우기

In [20]:
# 맥이나 리눅스의 경우 파일 확장자가 없다. 윈도우의 경우 exe 확장자까지 명시해야 한다.
driver = webdriver.Chrome('/Users/LimSH/Downloads/chromedriver_win32/chromedriver.exe', chrome_options=options)
# 모든 동작마다 크롬브라우저가 준비될 때 까지 최대 5초씩 대기
driver.implicitly_wait(5)

###로그인 없이 바로 #턱스크 검색한 웹 브라우저 띄우는 경우###
# 몇번 시도하면 막히는거 같음..
# driver.get("https://www.instagram.com/explore/tags/턱스크")
# # 브라우저가 표시될 때 까지 3초간 프로그램 대기
# time.sleep(3)

C:\Users\LimSH\anaconda3\envs\pydatavenv\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


## 로그인 & 턱스크 검색

In [21]:
### 로그인 하는 경우 ###

insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(3)

# 개인 계정정보 입력 후 로그인
login_button_element = driver.find_element_by_css_selector('#react-root > section > main > article > div > div > div > div:nth-child(2) > button')
login_button_element.click()
id_element = driver.find_element_by_css_selector('#loginForm > div.Igw0E.IwRSH.eGOV_._4EzTm.kEKum > div:nth-child(3) > div > label > input')
id_element.send_keys('아이디')
pw_element = driver.find_element_by_css_selector('#loginForm > div.Igw0E.IwRSH.eGOV_._4EzTm.kEKum > div:nth-child(4) > div > label > input')
pw_element.send_keys('비밀번호')
login_element = driver.find_element_by_css_selector('#loginForm > div.Igw0E.IwRSH.eGOV_._4EzTm.kEKum > div:nth-child(6) > button > div')
login_element.click()
driver.implicitly_wait(6)
popup_element = driver.find_element_by_css_selector('#react-root > section > main > div > div > div > button')
popup_element.click()
driver.implicitly_wait(5)
popup_element = driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm')
popup_element.click()

# 해시태그(#턱스크) 검색
search_element = driver.find_element_by_css_selector('#react-root > section > nav.NXc7H.f11OC > div > div > div.KGiwt > div > div > div:nth-child(2) > a > svg')
search_element.click()
search_element = driver.find_element_by_css_selector('#react-root > section > nav.gW4DF > div > header > div > h1 > div > div > div > div > label > input')
search_element.send_keys('턱스크')
choice_element = driver.find_element_by_css_selector('#react-root > section > main > div > div > ul > li:nth-child(1) > a > div')
choice_element.click()

## 이미지 가져와서 리스트에 저장

In [ ]:
import urllib.request 
from urllib import parse 
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
from selenium import webdriver 
import time 
import ssl

count = 0
img_list = []
while True :
    html = driver.page_source #driver의 페이지 소스를 가져와서 html 에 담음 
    soup =BeautifulSoup(html)

    insta = soup.select('.v1Nh3.kIKUG._bz0w') #이미지 파일이 들어있는 클래스(.으로 구분해준다) 
    print(len(insta))
                
    for i in insta :
        imgUrl = i.select_one('.KL4Bh').img['src'] #이미지가 들어있는 태그의 실질적 img 
        img_list.append(imgUrl)
        img_list = list(set(img_list)) #동일한 항목에 대한 중복제거
        
    try :
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤을 맨 아래로 이동
        time.sleep(2)
        count += 1
    
    except :        
        break
        
print("End")

In [23]:
print(len(img_list))

1491


In [25]:
datetime = dt.datetime.now().strftime("%y%m%d_%H%M")
dirname = "insta_%s" % (datetime)

In [26]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
session = requests.Session()
session.headers.update( {'User-agent': user_agent, 'referer': None} )

## 폴더 생성 & 이미지 저장

In [ ]:
# 저장된 이미지 수를 카운트하기 위한 변수
count = 0

# 폴더 생성하기
if not os.path.exists(dirname):
    os.mkdir(dirname)

# 이미지 URL수 만큼 반복
for image_url in img_list:

    count += 1

    # 파일이 저장될 경로 생성
    path = "%s/%04d.jpg" % (dirname, count)

    # 예외처리를 동반한 이미지 다운로드
    try:
        # 이미지 주소를 다운로드를 위해 stream 모드로 가져온다.
        r = session.get(image_url, stream=True)

        # HTTP 상태코드가 성공을 의미하는 값이 아니라면 에러로 간주하고 except 블록으로 강제 이동
        if r.status_code != 200:
            raise Exception
            
        # 추출한 데이터를 바이너리(이진값) 쓰기 모드로 저장 -> 'wb'
        with open(path, 'wb') as f:
            f.write(r.raw.read())
            print( "[Ok] %s(이)가 저장되었습니다." % path )
        
    except:
        # 이미지 다운로드 실패시 다음 이미지를 시도하기 위해 반복의 조건식으로 이동함
        print( "[Error] %s(은)는 저장에 실패했습니다." % path )
        continue

print('저장 완료')